<a href="https://colab.research.google.com/github/HAAIL/SavPytorchModel/blob/main/PyTorchModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.10.0
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip install iterative-stratification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
print("Data loading...")

import pandas as pd
import numpy as np
import re
import string as strinG

raw_data = pd.concat(map(pd.read_csv, ['goemotions_1.csv'])).to_numpy()
keys =  [
    
    'anger', #11

    'disgust', #20

    'fear', #23
   
    'joy', #26
  
    'sadness', #34
    'surprise', #35
    'neutral' #36
   

]
print(raw_data[0])
data = np.delete(raw_data[0:], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, \
                                16, 17, 18, 19, 21, 22, 24, 25, 27, 28, 29, \
                                30, 31, 32, 33], 1)


print(data[0])

def preprocess(string):
  
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    string = url_pattern.sub(r'', string) 
    string = re.sub("\S*@\S*\s?", "", string)   
    string = re.sub("\s+", " ", string)       
    string = string.translate(str.maketrans('', '', strinG.punctuation)) 
    string = string.lower()
    
    return string


X = []
y = []

for i in range(len(data)):
    if(data[i][-1] != 1) and preprocess(data[i][0]) != "": 
        X.append(preprocess(data[i][0]))
        y.append(data[i][1:])
X_save = X
X = np.array(X).astype(str)
y = np.array(y).astype(float)

        

print("Data loaded.")


Data loading...
['That game hurt.' 'eew5j0j' 'Brdd9' 'nrl' 't3_ajis4z' 't1_eew18eq'
 1548381039.0 1 False 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0]
['That game hurt.' 0 0 0 0 1 0 0]
Data loaded.


In [ ]:
import torch
!pip install iterative-stratification

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)


for train_index, test_index in msss.split(X, y):
   x_train, x_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)

for train_index, test_index in msss.split(x_test, y_test):
   x_val, x_test = X[train_index], X[test_index]
   y_val, y_test = y[train_index], y[test_index]

print(type(x_train))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
<class 'numpy.ndarray'>


In [ ]:
!pip install -c huggingface transformers==4.14.1 tokenizers==0.10.3 -y
!pip install huggingface_hub
!pip install -U sentence-transformers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import torch

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')    # Download vocabulary from S3 and cache.

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [ ]:

model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
config = torch.hub.load('huggingface/pytorch-transformers', 'config', 'bert-base-uncased')
config.output_attentions = True
config.output_hidden_states = True
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased', config=config)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification

In [ ]:

from collections import defaultdict


tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

'''
tokens = defaultdict(list)
for text in X:
  t = tokenizer(text.split(), is_split_into_words=True, padding='max_length', truncation=True, max_length=300)
  tokens['input_ids'].append(t['input_ids'])
  tokens['token_type_ids'].append(t['token_type_ids'])
  tokens['attention_mask'].append(t['attention_mask'])
  t['labels'] = list(map(lambda x: 1, t.word_ids())) # fake labels to simplify example
  tokens['labels'].append(t['labels'])

dataset = tf.data.Dataset.from_tensor_slices(dict(tokens))'''


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


"\ntokens = defaultdict(list)\nfor text in X:\n  t = tokenizer(text.split(), is_split_into_words=True, padding='max_length', truncation=True, max_length=300)\n  tokens['input_ids'].append(t['input_ids'])\n  tokens['token_type_ids'].append(t['token_type_ids'])\n  tokens['attention_mask'].append(t['attention_mask'])\n  t['labels'] = list(map(lambda x: 1, t.word_ids())) # fake labels to simplify example\n  tokens['labels'].append(t['labels'])\n\ndataset = tf.data.Dataset.from_tensor_slices(dict(tokens))"

In [ ]:

from datasets import load_dataset

dataset = load_dataset("go_emotions")
dataset["train"][100]


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
print(small_test_dataset[1])

  0%|          | 0/3 [00:00<?, ?it/s]

{'text': 'It hasn’t happened yet. Just talk to the supervisor or HR. In the meantime don’t borrow trouble.', 'labels': [5], 'id': 'efeu9xi', 'input_ids': [101, 1135, 8186, 787, 189, 2171, 1870, 119, 2066, 2037, 1106, 1103, 16014, 1137, 145, 2069, 119, 1130, 1103, 13115, 1274, 787, 189, 20055, 3819, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:


class MyModel(torch.nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()

        self.embedding = torch.nn.Embedding(20000, 300)
        self.convtranspose = torch.nn.ConvTranspose1d(7, 307, 3)
        self.activation1 = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool1d(2)
        #flatten
        self.dropout1 = torch.nn.Dropout(0.5)
        self.linear1 = torch.nn.LazyLinear(100)
        self.activation2 = torch.nn.Tanh()
        self.dropout2 = torch.nn.Dropout(0.2)
        self.linear2 = torch.nn.LazyLinear(7)
        self.activation3 = torch.nn.Sigmoid()

        

    def forward(self, x):
        
        x = self.embedding(x)
        x =self.convtranspose(x)
        x =self.activation1(x)
        x = self.maxpool(x)
        x = torch.flatten(x)
        x = self.dropout1(x)
        x = self.linear1(x)
        x = self.activation2(x)
        x = self.dropout2(x)
        x = self.linear2(x)
        x = self.activation3(x)
        return x

mymodel = MyModel()

print('The model:')
print(mymodel)



The model:
MyModel(
  (embedding): Embedding(20000, 300)
  (convtranspose): ConvTranspose1d(7, 307, kernel_size=(3,), stride=(1,))
  (activation1): ReLU()
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.5, inplace=False)
  (linear1): LazyLinear(in_features=0, out_features=100, bias=True)
  (activation2): Tanh()
  (dropout2): Dropout(p=0.2, inplace=False)
  (linear2): LazyLinear(in_features=0, out_features=7, bias=True)
  (activation3): Sigmoid()
)


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
loss_function = torch.nn.functional.binary_cross_entropy  

optimizer = torch.optim.Adam(mymodel.parameters(),lr=0.0001)  

epochs = 20

final_losses = []


for i in range(epochs):

    i= i+1

    y_pred = mymodel.forward(dataset)

    loss = loss_function(y_pred,y_train)

    final_losses.append(loss)



    print("Epoch number: {} and the loss : {}".format(i,loss.item()))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

TypeError: ignored